## Setup

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.metrics import precision_score, recall_score, make_scorer
from sklearn.model_selection import TimeSeriesSplit, GroupKFold, GridSearchCV, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from joblib import dump, load

#from utils import precision_at_k, recall_at_k

In [64]:
from utils import precision_at_k, recall_at_k

## Data Loading

In [2]:
X_train_f = pd.read_csv("./output/X_train_f.csv") # 2010 - 2014, w. protected attributes
X_train_s = pd.read_csv("./output/X_train_s.csv") # 2010 - 2014, w/o protected attributes
y_train = pd.read_csv("./output/y_train.csv").iloc[:,0]

X_calib_f = pd.read_csv("./output/X_calib_f.csv") # 2015, w. protected attributes
X_calib_s = pd.read_csv("./output/X_calib_s.csv") # 2015, w/o protected attributes
y_calib = pd.read_csv("./output/y_calib.csv").iloc[:,0]

X_test_f = pd.read_csv("./output/X_test_f.csv")
X_test_s = pd.read_csv("./output/X_test_s.csv")
y_test = pd.read_csv("./output/y_test.csv").iloc[:,0]

## Correlation Analysis

In [3]:
# Computes the absolute value of the correlation matrix for the training features with protected attributes
corrM = X_train_f.corr().abs() # Corr matrix of X
corrM = corrM.unstack() # flatten
corrMo = corrM.sort_values(kind = "quicksort") # sort correlations
corrMo[corrMo < 1].tail(20) # Filters out the self-correlations (which equal 1) and prints the last 20 entries (lowest correlations)

# to spot which features (including protected‐attribute proxies) are most strongly correlated, so I can 
# watch out for multicollinearity or fairness-related leakage

ft_tot_dur_byage        ft_tot_dur                0.954161
ft_tot_dur              ft_tot_dur_byage          0.954161
maxbula.Missing.        lastjob_pt99999           0.954524
lastjob_pt99999         maxbula.Missing.          0.954524
seeking1_tot_dur_byage  seeking1_tot_dur          0.955420
seeking1_tot_dur        seeking1_tot_dur_byage    0.955420
lastjob_none            maxbula.Missing.          0.961120
maxbula.Missing.        lastjob_none              0.961120
lastjob_type99999       maxbula.Missing.          0.961120
maxbula.Missing.        tsince_lm_contact_cat5    0.961120
tsince_lm_contact_cat5  maxbula.Missing.          0.961120
maxbula.Missing.        lastjob_type99999         0.961120
                        lastjob_parallel99999     0.961120
lastjob_parallel99999   maxbula.Missing.          0.961120
emp1_total_dur          emp1_total_dur_byage      0.963495
emp1_total_dur_byage    emp1_total_dur            0.963495
secjob_tot_dur          secjob_tot_dur_byage      0.9658

In [59]:
tscv = TimeSeriesSplit(4) # Create splits by year

In [60]:
for train_index, test_index in tscv.split(X_train_f):
    print("TRAIN:", train_index, "TEST:", test_index)

TRAIN: [   0    1    2 ... 4997 4998 4999] TEST: [5000 5001 5002 ... 9997 9998 9999]
TRAIN: [   0    1    2 ... 9997 9998 9999] TEST: [10000 10001 10002 ... 14997 14998 14999]
TRAIN: [    0     1     2 ... 14997 14998 14999] TEST: [15000 15001 15002 ... 19997 19998 19999]
TRAIN: [    0     1     2 ... 19997 19998 19999] TEST: [20000 20001 20002 ... 24997 24998 24999]


In [73]:
custom_precision25 = make_scorer(precision_at_k, needs_proba = True, k = 0.25) # Precision at top 25%
custom_precision10 = make_scorer(precision_at_k, needs_proba = True, k = 0.10) # Precision at top 10%

custom_recall25 = make_scorer(recall_at_k, needs_proba = True, k = 0.25) # Recall at top 25%
custom_recall10 = make_scorer(recall_at_k, needs_proba = True, k = 0.10) # Recall at top 10%

In [74]:
score = {'log_loss': 'neg_log_loss',
         'auc': 'roc_auc',
         'precision': 'precision', # uses default model threshold of 0.5
         'recall': 'recall',
         'precision_at_k25': custom_precision25, # uses custom threshold 
         'recall_at_k25': custom_recall25,
         'precision_at_k10': custom_precision10,
         'recall_at_k10': custom_recall10}

## 01 Logit Regression (w. protected attributes)

In [ ]:
glm1 = LogisticRegression(penalty = None, solver = 'lbfgs', max_iter = 1000)
glm1.fit(X_train_f, y_train)

/Users/julia/Desktop/CMA_Fairness/cma_f/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=10000, penalty=None)

In [75]:
glmcv1 = cross_validate(estimator = glm1, 
                       X = X_train_f,
                       y = y_train,
                       cv = tscv,
                       n_jobs = -1, # use all available cores
                       scoring = score)

/Users/julia/Desktop/CMA_Fairness/cma_f/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/julia/Desktop/CMA_Fairness/cma_f/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:978: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/julia/Desktop/CMA_Fairness/cma_f/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 140, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "/Us

In [5]:
coefs1 = pd.DataFrame(X_train_f.columns, columns = ['var'])
coefs1['coef'] = pd.DataFrame(glm1.coef_).transpose()

# Build a DataFrame of feature names + their learned coefficients, to inspect which variables 
# (including protected attrs) the model weights most heavily.

In [6]:
dump(glm1, './models/glm1.joblib')

['./models/glm1.joblib']

## Predict

In [7]:
k75 = 0.75 # Top 75% 
k25 = 0.25 # Top 25% 
k10 = 0.1 # Top 10%

# Logit

In [26]:
glm1_p = glm1.predict_proba(X_test_f)[:,1] # glm1

# Generate the predicted probability of the positive class for each test sample

In [27]:
threshold75 = np.sort(glm1_p)[::-1][int(k75*len(glm1_p))]
threshold25 = np.sort(glm1_p)[::-1][int(k25*len(glm1_p))]
threshold10 = np.sort(glm1_p)[::-1][int(k10*len(glm1_p))] # threshold10 is the score above which only the top 10% of test samples lie

In [10]:
glm1_c1 = glm1_p.copy()
glm1_c1[glm1_c1 < threshold10] = 0
glm1_c1[glm1_c1 >= threshold10] = 1

# Create a binary classification vector where only the top 10% by predicted probability are labeled “1”

In [11]:
glm1_c2 = glm1_p.copy()
glm1_c2[glm1_c2 < threshold25] = 0
glm1_c2[glm1_c2 >= threshold25] = 1

In [12]:
glm1_c3 = glm1_p.copy()
glm1_c3[(glm1_c3 <= threshold75) | (glm1_c3 >= threshold25)] = 0
glm1_c3[(glm1_c3 > threshold75) & (glm1_c3 < threshold25)] = 1

## Performance evaluation

In [13]:
from sklearn.metrics import accuracy_score, f1_score, classification_report

In [14]:
for preds, label in zip(
    [glm1_c1, glm1_c2, glm1_c3],
    ["Top 10%", "Top 25%", "Middle 25–75%"]
):
    acc = accuracy_score(y_test, preds)
    f1  = f1_score(y_test, preds)
    print(f"{label:15s} → Accuracy: {acc:.3f},  F1-score: {f1:.3f}")

Top 10%         → Accuracy: 0.836,  F1-score: 0.281
Top 25%         → Accuracy: 0.746,  F1-score: 0.329
Middle 25–75%   → Accuracy: 0.483,  F1-score: 0.177


## Combine and save

In [15]:
preds_test = pd.concat([pd.DataFrame(np.array(y_test), columns = ['y_test']),
                         pd.DataFrame(glm1_p, columns = ['glm1_p']),
                         pd.DataFrame(glm1_c1, columns = ['glm1_c1']),
                         pd.DataFrame(glm1_c2, columns = ['glm1_c2']),
                         pd.DataFrame(glm1_c3, columns = ['glm1_c3'])],
                        axis = 1)

'''
Build a single DataFrame side by side with:
      - The true labels (‘y_test’)
      - The raw predicted probabilities (‘glm1_p’)
      - Each binary decision vector at different cutoffs (‘glm1_c1’, ‘glm1_c2’, ‘glm1_c3’).
'''

'\nBuild a single DataFrame side by side with:\n      - The true labels (‘y_test’)\n      - The raw predicted probabilities (‘glm1_p’)\n      - Each binary decision vector at different cutoffs (‘glm1_c1’, ‘glm1_c2’, ‘glm1_c3’).\n'

In [16]:
preds_test.to_csv('./output/preds_test.csv', index = False)

## Conformal

In [17]:
# Miscoverage level
alpha = 0.1

In [18]:
# Conformal wrapper functions
def compute_nonconformity_scores(probs: np.ndarray, labels: np.ndarray) -> np.ndarray:
    """
    For each calibration example, return 1 - p_true.
    probs: array of shape (n_samples, 2)
    labels: array of shape (n_samples,) with values in {0, 1}
    """
    p_true = probs[np.arange(len(labels)), labels]
    return 1.0 - p_true

## ??? Alternative scoring function negative log-probabilites?? 


def find_threshold(nonconformity: np.ndarray, alpha: float) -> float:
    """
    Return the (1-alpha)-quantile (higher interpolation) of the nonconformity scores.
    """
    return np.quantile(nonconformity, 1 - alpha, interpolation='higher')


def predict_conformal_sets(model, X: pd.DataFrame, q_hat: float) -> np.ndarray:
    """
    For each row in X, compute the conformal prediction set.
    Returns a list of sets.
    """
    probs = model.predict_proba(X) # shape (n, 2)
    nonconf_matrix = 1.0 - probs # shape (n, 2): nonconf score for label = 0, 1
    # include c whenver nonconf_matrix[i,c] <= q_hat
    return [set(np.where(nc_row <= q_hat)[0]) for nc_row in nonconf_matrix]


def evaluate_sets(pred_sets: list, y_true: pd.Series) -> dict:
    """
    Compute empirical coverage and average set size.
    """
    coverage = np.mean([
        (y_true.iloc[i] in pred_sets[i])
        for i in range(len(y_true))
    ])
    avg_size = np.mean([len(s) for s in pred_sets])
    return {
        'coverage': coverage,
        'avg_size': avg_size
    }


In [19]:
probs_calib = glm1.predict_proba(X_calib_f)

In [20]:
nc_scores = compute_nonconformity_scores(probs_calib, y_calib)

In [21]:
q_hat = find_threshold(nc_scores, alpha)

/var/folders/h7/6qcvyjh51cg86vrxn3xs8c_40000gn/T/ipykernel_2779/3779898564.py:1: DeprecationWarning: the `interpolation=` argument to quantile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review the method they used. (Deprecated NumPy 1.22)
  q_hat = find_threshold(nc_scores, alpha)


In [46]:
# With test data
pred_sets = predict_conformal_sets(glm1, X_test_f, q_hat)

In [47]:
# With test data
evaluation = evaluate_sets(pred_sets, y_test)
print(f"Coverage: {evaluation['coverage']:.2f}")
print(f"Avg. set size: {evaluation['avg_size']:.2f}")

Coverage: 0.90
Avg. set size: 1.13


## Analyzing CP per group

In [56]:
# Ensure pred_sets index aligns with the calibration data index
pred_sets = pred_sets.copy()  # Work on a copy to avoid modifying original accidentally
try:
    pred_sets.index = X_test_f.index  # Align index to calibration data's index
except Exception as e:
    print("Index alignment not applied (perhaps pred_sets already has correct index):", e)

# Now the indices of pred_sets should correspond to the same individuals as cp_groups.


Index alignment not applied (perhaps pred_sets already has correct index): 'list' object attribute 'index' is read-only


In [48]:
cp_groups = pd.DataFrame({'pred_set': pred_sets})

In [49]:
cp_groups['pred_set'] = cp_groups['pred_set'].apply(lambda s: {int(x) for x in s})

In [50]:
cp_groups['true_label'] = y_test

In [51]:
cp_groups['frau1'] = X_test_f['frau1'].values

In [52]:
cp_groups['nongerman'] = np.where(X_test_f['maxdeutsch1'] == 0, 1, 0) # creates new column 'nongerman', if maxdeutsch1 == 0 then nongerman = 1, else 0 
cp_groups.loc[X_test_f['maxdeutsch.Missing.'] == 1, 'nongerman'] = np.nan # overwrite nongerman with NaN for any row where maxdeutsch.Missing. equals 1
cp_groups['nongerman_male'] = np.where((cp_groups['nongerman'] == 1) & (cp_groups['frau1'] == 0), 1, 0)
cp_groups['nongerman_female'] = np.where((cp_groups['nongerman'] == 1) & (cp_groups['frau1'] == 1), 1, 0)

In [53]:
# Filter to ambiguous cases
ambig = cp_groups[cp_groups['pred_set'].apply(lambda s: set(s) == {0, 1})]

In [54]:
n_total = len(ambig)

n_true_label_1      = (ambig['true_label'] == 1).sum() / n_total
n_frau1_1           = (ambig['frau1'] == 1).sum() / n_total
n_nongerman_1       = (ambig['nongerman'] == 1).sum() / n_total
n_nongerman_male_1  = (ambig['nongerman_male'] == 1).sum() / n_total
n_nongerman_female_1= (ambig['nongerman_female'] == 1).sum() / n_total


In [55]:
print("Among ambiguous cases (pred_set == {0,1}):")
print(f"true_label == 1:        {n_true_label_1}")
print(f"frau1 == 1:             {n_frau1_1}")
print(f"nongerman == 1:         {n_nongerman_1}")
print(f"nongerman_male == 1:    {n_nongerman_male_1}")
print(f"nongerman_female == 1:  {n_nongerman_female_1}")

Among ambiguous cases (pred_set == {0,1}):
true_label == 1:        0.297811110142147
frau1 == 1:             0.47946280631377
nongerman == 1:         0.09209034621086597
nongerman_male == 1:    0.03191767681172059
nongerman_female == 1:  0.060172669399145375
